In [1]:
%load_ext watermark
%watermark

2017-12-06T17:32:59+01:00

CPython 3.6.1
IPython 6.2.1

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.10.0-40-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10, 10]
np.random.seed(42)

# Optimización de hiperparámetros

Hasta ahora hemos visto una manera relativamente sencilla de ver que valores de los hiperparámetros funcionan mejor, mediante las curvas de validación.

Estas curvas son muy útiles para darnos información a los Data Scientists, pero tienen dos problemas:
- Son métodos gráficos, esto significa que necesitan un humano para interpretarlas y no nos permiten automatizar el proceso para encontrar los hiperparámetros óptimos.
- Solo toman un hiperparámetro a la vez. Esto significa que hacen que sea más dificil el evaluar combinaciones de los hiperparámetros (si quisieramos evaluar multiples hiperparámetros tendriamos que hacer gráficas de planos o hiperplanos).

Vamos a ver ahora métodos más robustos para dado un modelo, encontrar el conjunto de hiperparámetros que hace que funcione mejor.

# Cargamos los datos

Vamos a usar un dataset nuevo, el [Census Income Dataset](https://archive.ics.uci.edu/ml/datasets/Census+Income). Es un dataset que tiene datos demográficos sobre 50,000 personas en Estados Unidos y como variable objetivo tiene una variable booleana (Verdadero/Falso) sobre si dicha persona gana más de 50K$ al año o no.

In [2]:
censo = pd.read_csv("data/salario_censo.csv")

In [3]:
censo.shape

(32561, 13)

In [4]:
censo.head()

,edad,clase_laboral,nivel_educativo,status_matrimonial,ocupacion,relacion,raza,genero,ganancias_capital,perdidas_capital,horas_laborables,pais_origen,objetivo
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Pipeline de procesamiento de datos

In [5]:
variable_dependiente = "objetivo"
variables_independientes = censo.drop(variable_dependiente, axis=1).columns
censo_X = censo[variables_independientes]
censo_y = censo[variable_dependiente]

In [6]:
censo_y.unique()

array([' <=50K', ' >50K'], dtype=object)

En este caso la variable objetivo está definida como texto, asi que la convertimos a una variable binaria numérica.

In [7]:
censo_y = censo_y.replace({" <=50K":0, " >50K":1})

Separamos datos en numéricos y no numéricos. Viendo el [diccionario de datos](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names) del dataset vemos que no hay variables categóricas, solo la variable educacion que ya viene codificada como numérica (*education-num*).

In [8]:
datos_numericos = censo_X.select_dtypes([int, float])
col_numericas = datos_numericos.columns

datos_categoricos = censo_X.select_dtypes([object])
col_no_numericas = datos_categoricos.columns

Vamos a usar un transformador nuevo `MultiLabelBinarizer`. Es como el LabelBinarizer pero funciona para multiples columnas

In [9]:
from sklearn.preprocessing import Imputer, StandardScaler, MultiLabelBinarizer

In [11]:
b = MultiLabelBinarizer()
b.fit_transform(
 [
     ["gato", "patata", "rojo"],
     ["perro", "zanahoria", "azul"],
     ["camello", "patata", "verde"],
     ["gato", "patata", "rojo"]
 ]
)

array([[0, 0, 1, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 1, 0, 0]])

Con `classes_` podemos ver los diferentes valores de las variables categoricas

In [12]:
b.classes_

array(['azul', 'camello', 'gato', 'patata', 'perro', 'rojo', 'verde',
       'zanahoria'], dtype=object)

Creamos el transformador `BinarizadorMultipleCategorico` que es básicamente el MultiLabelBinarizer pero "arreglado" para que funcione en Pipelines (Estoy usando sklearn 0.19.0, este bug se arreglará en el futuro).

In [13]:
from sklearn.base import TransformerMixin
from sklearn import preprocessing

class ColumnExtractor(TransformerMixin):
    """Transformador que selecciona columnas de un dataframe"""
    def __init__(self, columns):
        self.columns = columns
        
    def transform(self, X, **transform_params):
        return X[self.columns].as_matrix()
        
    def fit(self, X, y=None, **fit_params):
        return self
    
class BinarizadorMultipleCategorico(preprocessing.MultiLabelBinarizer):
    def fit(self, X, y=None):
        super(BinarizadorMultipleCategorico, self).fit(X)
        
    def transform(self, X, y=None):
        return super(BinarizadorMultipleCategorico, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(BinarizadorMultipleCategorico, self).fit(X).transform(X)

In [14]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [15]:
pipeline_numerico = Pipeline([
    ('selector_numerico', ColumnExtractor(columns=col_numericas)),
    ('imputador', Imputer()),
    ('escalador', StandardScaler()),
])

pipeline_categorico = Pipeline([
    ('selector_categorico', ColumnExtractor(columns=col_no_numericas)),
    ('codificador_numerico', BinarizadorMultipleCategorico()),
])

In [16]:
pipeline_categorico.fit_transform(censo_X).shape

(32561, 84)

In [18]:
col_numericas

Index(['edad', 'nivel_educativo', 'ganancias_capital', 'perdidas_capital',
       'horas_laborables'],
      dtype='object')

In [17]:
pipeline_numerico.fit_transform(censo_X).shape

(32561, 5)

In [19]:
pipeline_procesado = FeatureUnion([
    ('transformacion_numericas', pipeline_numerico),
    ('transformacion_categorica', pipeline_categorico),
])

In [20]:
pipeline_procesado

FeatureUnion(n_jobs=1,
       transformer_list=[('transformacion_numericas', Pipeline(memory=None,
     steps=[('selector_numerico', <__main__.ColumnExtractor object at 0x7f8ddb4420b8>), ('imputador', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('escalador', StandardScaler(copy=True, with_mean=...28>), ('codificador_numerico', BinarizadorMultipleCategorico(classes=None, sparse_output=False))]))],
       transformer_weights=None)

In [21]:
censo_X_procesado = pipeline_procesado.fit_transform(censo_X)

In [22]:
censo_X_procesado.shape

(32561, 89)

Antes que nada vamos a ver que puntuaciones tienen unos cuantos modelos con sus  hiperparámetro por defecto

In [ ]:
resultados = {}

In [24]:
from sklearn.model_selection import cross_validate

def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring="roc_auc", n_jobs=-1, cv=5, return_train_score=True)
    return resultados_estimador

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].max()
    return resultados_df

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
resultados["reg_logistica"] = evaluar_modelo(LogisticRegression(), censo_X_procesado, censo_y)
resultados["naive_bayes"] = evaluar_modelo(GaussianNB(), censo_X_procesado, censo_y)
resultados["rf"] = evaluar_modelo(RandomForestClassifier(), censo_X_procesado, censo_y)
resultados["svc"] = evaluar_modelo(SVC(), censo_X_procesado, censo_y)

In [26]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
naive_bayes,0.061415,0.011542,0.781202,0.783341,0.000949,0.001289,0.859760,0.786943
reg_logistica,0.394294,0.021813,0.906715,0.908326,0.006094,0.002436,0.997894,0.912503
rf,0.389938,0.017129,0.870704,0.995423,0.006027,0.001913,0.958262,1.000000
svc,64.697272,8.953962,0.908628,0.910540,1.000000,1.000000,1.000000,0.914727


Vamos a seleccionar un estimador en función de los resultados iniciales y optimizarlo. Elijo el estimador Random Forest por que funciona muy bien en comparación a los demás y es bastánte rápido de entrenar.

In [29]:
estimador_rf = RandomForestClassifier()

Scikit-learn tiene dos métodos de optimización de hiperparámetros, [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) y [RandomizedSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV).

`GridSearchCV` funciona realizando una busqueda en una malla, es decir, pasandole un conjunto de posibles opciones de hiperparámetros evalua de forma completa cada combinación de dichos parámetros (es decir, el valor 1 del hiperparámetro 1 combinado con todos los posibles valores de los demás hiperparámetros, el valor 2 del hiperparámetro 1 combinado con todos los posibles valores de los demás hiperparámetros, etcétera).

La ventaja de utilizar una búsqueda de malla es que nos aseguramos de que se han probado todas las combinaciones posibles. El problema es que el proceso requiere mucho tiempo de computación, y según que dataset usemos 

In [26]:
%%timeit
import time
def foo():
    time.sleep(1)

295 ns ± 15.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [27]:
%%timeit -n 1 -r 1  #n 1 dice que ejecute esta celda solo una vez, -r 1 que ejecute un solo loop
def foo():
    time.sleep(1)

869 ns ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [34]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [260]:
print(estimador_rf.__doc__)

A random forest classifier.

    A random forest is a meta estimator that fits a number of decision tree
    classifiers on various sub-samples of the dataset and use averaging to
    improve the predictive accuracy and control over-fitting.
    The sub-sample size is always the same as the original
    input sample size but the samples are drawn with replacement if
    `bootstrap=True` (default).

    Read more in the :ref:`User Guide <forest>`.

    Parameters
    ----------
    n_estimators : integer, optional (default=10)
        The number of trees in the forest.

    criterion : string, optional (default="gini")
        The function to measure the quality of a split. Supported criteria are
        "gini" for the Gini impurity and "entropy" for the information gain.
        Note: this parameter is tree-specific.

    max_features : int, float, string or None, optional (default="auto")
        The number of features to consider when looking for the best split:

        - If int, th

In [30]:
estimador_rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

Vamos a definir los límites de la búsqueda de hiperparámetros.

In [31]:
np.linspace(10,1000,10).astype(int)

array([  10,  120,  230,  340,  450,  560,  670,  780,  890, 1000])

In [32]:
parametros_busqueda_rf = {
    "criterion": ["gini", "entropy"],
    "n_estimators": np.linspace(10,1000,10).astype(int),
    "class_weight": [None, "balanced"]
}

In [35]:
grid = GridSearchCV(estimator=estimador_rf, 
                    param_grid=parametros_busqueda_rf,
                    scoring="roc_auc", n_jobs=-1)

`GridSearchCV` se comporta como un estimador en cuanto a que tiene un metodo fit que usamos para "entrenarlo" y que realize la búsqueda en malla.

Para ver cuanto tiempo tarda en realizar la búsqueda usamos la mágia de Jupyter notebook `%%timeit` que evalua el tiempo que tarda una función en ejecutarse


In [83]:
%%timeit -n 1 -r 1
grid.fit(censo_X_procesado, censo_y)

59.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


En mi ordenador la busqueda en malla ha tardado 7minutos y 49 segundos 

Ahora podemos ver la puntuación que ha obtenido el mejor estimador así como los parámetros del mismo

In [69]:
print(grid.best_score_)
print(grid.best_estimator_)

0.89726431782
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=890, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


Tras haberlo ajustado, Gridsearch nos devuelve el ranking de todas las variantes evaluadas junto con métricas de su funcionamiento con el atributo `cv_results_`

In [64]:
pd.DataFrame(grid.cv_results_).sort_values(by="rank_test_score")

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_class_weight,param_criterion,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
18,48.395469,3.825982,0.897264,0.998138,None,entropy,890,"{'class_weight': None, 'criterion': 'entropy',...",1,0.894322,0.998229,0.896448,0.998153,0.901023,0.998032,1.722333,0.315958,0.002796,0.000081
16,37.419543,2.961567,0.897248,0.998130,None,entropy,670,"{'class_weight': None, 'criterion': 'entropy',...",2,0.894501,0.998228,0.896451,0.998141,0.900792,0.998021,1.058009,0.130893,0.002629,0.000085
19,56.035860,3.788953,0.897244,0.998139,None,entropy,1000,"{'class_weight': None, 'criterion': 'entropy',...",3,0.894072,0.998229,0.896487,0.998146,0.901173,0.998042,1.453320,0.025295,0.002948,0.000076
17,42.341748,3.100389,0.897123,0.998136,None,entropy,780,"{'class_weight': None, 'criterion': 'entropy',...",4,0.894375,0.998222,0.896254,0.998156,0.900739,0.998031,1.235332,0.239111,0.002669,0.000079
15,29.182102,2.295158,0.897005,0.998133,None,entropy,560,"{'class_weight': None, 'criterion': 'entropy',...",5,0.894167,0.998228,0.896365,0.998151,0.900484,0.998021,0.463407,0.318522,0.002618,0.000085
14,24.451602,1.752896,0.896987,0.998122,None,entropy,450,"{'class_weight': None, 'criterion': 'entropy',...",6,0.894299,0.998227,0.896358,0.998126,0.900303,0.998014,0.518100,0.179451,0.002491,0.000087
8,39.529309,3.353667,0.896701,0.998138,None,gini,890,"{'class_weight': None, 'criterion': 'gini', 'n...",7,0.893811,0.998224,0.895982,0.998142,0.900310,0.998048,2.404425,0.397396,0.002701,0.000072
9,48.145448,3.716241,0.896669,0.998139,None,gini,1000,"{'class_weight': None, 'criterion': 'gini', 'n...",8,0.893764,0.998226,0.896117,0.998144,0.900128,0.998045,2.437924,0.189916,0.002627,0.000074
7,33.959718,2.785672,0.896637,0.998135,None,gini,780,"{'class_weight': None, 'criterion': 'gini', 'n...",9,0.893648,0.998226,0.895913,0.998130,0.900351,0.998050,0.768166,0.143435,0.002784,0.000072
36,34.654465,2.194206,0.896564,0.997393,balanced,entropy,670,"{'class_weight': 'balanced', 'criterion': 'ent...",10,0.894296,0.997597,0.895903,0.997535,0.899494,0.997047,0.568587,0.067388,0.002173,0.000246


`GridSearchCV` al estar ajustado se convierte en un estimador, por lo cual podemos usar el método predict, por debajo simplemente se usará el mejor estimador `grid.best_estimator_`. 

Para añadir el funcionamiento del mejor estimador obtenido por el modelo con nuestra funcion `evaluar_modelo` no usamos el objeto grid en si, ya que la funcion `cross_validate` hace multiples ajustes y evaluaciones (volveriamos a esperar los 8 minutos que a tardado un ajuste multiplicado por el número de validaciones cruzadas!).

Para evaluar el funcionamiento del mejor estimador simplemente usamos la funcion con el mejor estimador directamente.

In [84]:
resultados["rf_gridsearch"] = evaluar_modelo(grid.best_estimator_, censo_X_procesado, censo_y)

Ahora vamos a realizar la misma optimización de parámetros pero usando `RandomizedSearchCV`. RandomizedSearchCV funciona de forma similar a GridSearchCV, pero en vez de evaluar todas las combinaciones posibles de hiperparámetros, se toman n muestras de hiperparámetros de dichas distribuciones.

Se recomienda usar distribuciones en vez de valores fijos para hiperparámetros continuos.

Primero vamos a evaluar el funcionamiento de la busqueda aleatoria con los mísmos hiperparámetros que hemos usado en la busqueda en malla. Para `RandomizedSearchCV` tenemos que indicarle cuantas variantes de hiperparámetros utilizar (definidas por el parámetro n_iter, por defecto toma 10 variantes). Dado que dicha búsqueda toma muestreos el parámetro ya no se llama `param_grid` sino `param_distributions`.

In [36]:
busqueda_random = RandomizedSearchCV(estimator=estimador_rf, 
                    param_distributions=parametros_busqueda_rf,
                   scoring="roc_auc", n_jobs=-1, n_iter=10)

In [ ]:
%%timeit -n 1 -r 1
busqueda_random.fit(censo_X_procesado, censo_y)

La búsqueda con 10 iteraciones ha tardado 1min 25s en mi máquina. Veamos como ha funcionado.

In [70]:
print(busqueda_random.best_score_)
print(busqueda_random.best_estimator_)

0.896926092788
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=340, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


La búsqueda de malla obtuvo un ROC AUC máximo de 0.89726431782 versus 0.896926092788 obtenido por la búsqueda aleatoria. Sin embargo la busqueda aleatoria ha tardado 8 veces menos!

In [87]:
resultados["rf_randomizedsearch"] = evaluar_modelo(grid.best_estimator_, censo_X_procesado, censo_y)

Una ventaja del Randomized Search es que nos permite evaluar un espacio de hiperparámetros más amplio para un tiempo de computación similar.

Para ver esto vamos a ampliar el espacio de búsqueda de hiperparámetros y hacer 100 muestreos.

In [40]:
from scipy.stats import randint as sp_randint

param_dist_random = {
    "max_depth": [3, None],
    "max_features": sp_randint(1, 11),
    "min_samples_split": sp_randint(2, 11),
    "min_samples_leaf": sp_randint(1, 11),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"],
    "n_estimators": np.linspace(10,1000,10).astype(int),
}

In [41]:
busqueda_random_100 = RandomizedSearchCV(estimator=estimador_rf, 
                    param_distributions=param_dist_random,
                   scoring="roc_auc", n_jobs=-1, n_iter=100)

In [109]:
%%timeit -n 1 -r 1
busqueda_random_100.fit(censo_X_procesado, censo_y)

14min 55s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


En mi máquina esta búsqueda ha tardado 8 minutos 54 segundos, un poco más que el grid search

In [96]:
print(busqueda_random_100.best_score_)
print(busqueda_random_100.best_estimator_)

0.91950285418
RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None, max_features=10,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=3,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)


La búsqueda aleatoria con los nuevos parámetros ha tardado un tiempo similar a la busqueda en malla, pero ha obtenido una puntuación máxima ROC AUC de 0.91950285418 (versus 0.89726431782 de la busqueda en malla)

In [110]:
resultados["rf_randomizedsearch_100"] = evaluar_modelo(busqueda_random_100.best_estimator_,
                                                      censo_X_procesado, censo_y)

Vemos que el estimador obtenido con la búsqueda aleatoria es el que mejor funciona.

En general, salvo que el espacio de hiperparámetros que queramos explorar sea pequeño, es mejor el utilizar `RandomizedSearchCV` en vez de `GridSearchCV`. Esto es así por que en general no existe un unico conjunto de hiperparámetros que obtiene el mejor funcionamiento, sino que suelen existir multiples "areas" en el espacio dimensional de los hiperparámetros que funcionan de forma similar. Al hacer una búsqueda aleatoria podemos explorar las diversas areas en un tiempo más reducido.

# Optimización de parámetros dentro de un Pipeline

Los algoritmos de busqueda de sklearn siguen la API de transformadores y estimadores. Esto significa que podemos crear un pipeline e incluir la optimización de hiperparámetros dentro.

In [43]:
busqueda_random_10 = RandomizedSearchCV(estimator=estimador_rf, 
                    param_distributions=param_dist_random,
                   scoring="roc_auc", n_jobs=-1, n_iter=10)

pipeline_estimador = Pipeline(
    [
     ("procesado", pipeline_procesado),
     ("estimador", busqueda_random_10)   
    ])

Ahora podemos ajustar directamente en los datos originales sin tener que preprocesarlos.

In [104]:
pipeline_estimador.fit(censo_X, censo_y)

Pipeline(memory=None,
     steps=[('procesado', FeatureUnion(n_jobs=1,
       transformer_list=[('transformacion_numericas', Pipeline(memory=None,
     steps=[('selector_numerico', <__main__.ColumnExtractor object at 0x7f0962060dd8>), ('imputador', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)),...', random_state=None, refit=True,
          return_train_score=True, scoring='roc_auc', verbose=0))])

In [107]:
pipeline_estimador.predict(censo_X)

array([0, 0, 0, ..., 0, 0, 1])

Hay varias librerias externas que permiten hacer optimización de parámetros de forma más flexible y/o compleja que las implementaciones que soporta scikit-learn por defecto.


# Scikit-optimize [(link)](https://scikit-optimize.github.io/)

Scikit-Optimize, o skopt, es una librería que implementa multiples metodos para optimizar hiperparámetros de forma secuencial.

Se instala con:

`{sys.executable} -m pip install scikit-optimize` (existe una version en conda-forge pero solo de una version antigua)

La api de scikit-optimize no es tan similar a la de sklearn como seria posible, no obstante es muy facil de usar. Permite usar diversos algoritmos para ayudar al proceso de optimización, por ejemplo procesos gausianos o Bosques aleatorios

In [46]:
import skopt

In [47]:
skopt.__version__

'0.4'

In [48]:
from skopt import gp_minimize 

En vez de usar un diccionario con el espacio de hiperparámetros que queremos buscar, scikit-optimize necesita pasarle una lista de parámetros.

skopt es una libreria relativamente nueva, y tiene ciertas limitaciones comparada con scikitlearn. Por ejemplo, en vez de diccionarios con los nombres de los parámetros,  espera como inputs listas, no se pueden usar funciones de distribuciones (como scipy.randint).

In [128]:
from skopt import space

param_espacio_skopt = [
    space.Integer(3, 10), #max_depth
    space.Integer(1, 11), #max_features
    (0.001, 0.99, "uniform"), #min_samples_split
    (0.001, 0.5, "uniform"), #min_samples_leaf
    space.Integer(1, 1000), #n_estimators
    space.Categorical(["gini", "entropy"]), #criterion,
    space.Categorical([True, False]), #bootstrap
]

Scikit-optimize necesita que definamos la funcion objetivo, que ira variando en funcion de los parámetros elegidos. Dicha función tiene que crear el estimador y evaluarlo y devolver la evaluación.

In [129]:
from sklearn.model_selection import cross_val_score

estimador_rf = RandomForestClassifier()

def funcion_optimizable(params):
    #params es simplemente una selección especifica de hiperparámetros
    max_depth, max_features, min_samples_split, min_samples_leaf,  n_estimators, criterion, bootstrap = params

    estimador_rf.set_params(
                   max_depth=max_depth,
                   max_features=max_features,
                   min_samples_split=min_samples_split, 
                   min_samples_leaf=min_samples_leaf,
                   n_estimators=n_estimators,
                   criterion=criterion
                  )

    return -np.mean(cross_val_score(estimador_rf, censo_X_procesado, censo_y, cv=5, n_jobs=-1,
                                    scoring="roc_auc"))

Ahora podemos dejar que skopt optimize los outputs de la funcion `funcion_optimizable` mediante el uso de `gp_minimize`

In [130]:
%%timeit -n 1 -r 1
resultado_gp = gp_minimize(funcion_optimizable, param_espacio_skopt, n_calls=100, random_state=42)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:366: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:366: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:366: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:366: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


25min 31s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


este proceso ha tardado 25 minutos en mi maquina

El parámetro `x` nos da el vector con los parámetros con mejor funcionamiento

In [134]:
resultado_gp.x

[7, 10, 2, 3, 46, 'gini']

In [132]:
estimador_skopt_gp_100 = RandomForestClassifier(
    max_depth=resultado_gp.x[0],
    max_features=resultado_gp.x[1],
    min_samples_split=resultado_gp.x[2], 
    min_samples_leaf=resultado_gp.x[3],
    n_estimators=resultado_gp.x[4],
    criterion=resultado_gp.x[5]
)

In [133]:
resultados["rf_skopt_gp_100"] = evaluar_modelo(estimador_skopt_gp_100, censo_X_procesado, censo_y)

skopt (version >0.4) tambien tiene una implementacion de `BayesSearchCV`, que es un reemplazo de GridSearchCV pero que usa optimización bayesiana (en vez de probar todas las posibilidades).

In [94]:
from skopt import BayesSearchCV 

param_espacio_skopt_bayesCV = {
     "max_depth": space.Integer(3, 10), #
    "max_features": space.Integer(1, 11), #
    "min_samples_split": space.Real(0.001, 0.99, "uniform"), #
    "min_samples_leaf": space.Real(0.001, 0.5, "uniform"), #
    "n_estimators": space.Integer(1, 1000), #
    "criterion": space.Categorical(["gini", "entropy"]),
    "boostrap": space.Categorical([True, False])
}

busqueda_bayesiano_skopt_100 = BayesSearchCV(
    estimator=estimador_rf, 
    search_spaces=param_espacio_skopt_bayesCV,
    scoring="roc_auc", n_jobs=-1, n_iter=100,
    random_state=42
)

In [95]:
%%timeit -n 1 -r 1
busqueda_bayesiano_skopt_100.fit(censo_X_procesado, censo_y)

10min 39s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [106]:
busqueda_bayesiano_skopt_100.best_estimator_.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 11,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 0.001,
 'min_samples_split': 0.001,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 212,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [98]:
resultados["rf_bayesiano_skopt_100"] = evaluar_modelo(busqueda_bayesiano_skopt_100.best_estimator_,
                                                      censo_X_procesado, censo_y)

In [135]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
naive_bayes,0.066678,0.014363,0.781202,0.783341,0.001043,0.010738,0.849714,0.786865
reg_logistica,0.460645,0.018298,0.906715,0.908326,0.007205,0.013680,0.986234,0.912413
rf,0.395003,0.017742,0.874312,0.995521,0.006179,0.013264,0.950989,1.000000
rf_bayesiano_skopt_100,7.136472,0.193855,0.910383,0.914396,0.111627,0.144932,0.990223,0.918510
rf_gridsearch,3.986320,0.236890,0.848783,0.849026,0.062353,0.177107,0.923221,0.852846
rf_randomizedsearch,4.077757,0.253586,0.854896,0.855122,0.063783,0.189589,0.929870,0.858970
rf_randomizedsearch_100,63.931399,1.337555,0.919371,0.958161,1.000000,1.000000,1.000000,0.962472
rf_skopt_gp_100,1.557633,0.057794,0.906729,0.910607,0.024364,0.043208,0.986249,0.914704


### Hyperopt-sklearn [(link)](https://github.com/hyperopt/hyperopt-sklearn)

Hyperopt-sklearn es una implementación de Hyperopt, que es la librería más famosa para optimización de hiperparámetros) pero preparado para funcionar con scikit-learn

In [139]:
import sys
!{sys.executable} -m pip install git+https://github.com/hyperopt/hyperopt-sklearn.git

  Cloning https://github.com/hyperopt/hyperopt-sklearn.git to /tmp/pip-hw66qmw5-build
    100% |████████████████████████████████| 102kB 1.1MB/s a 0:00:01
  Using cached nose-1.3.7-py3-none-any.whl
    100% |████████████████████████████████| 378kB 1.7MB/s ta 0:00:01
    100% |████████████████████████████████| 1.6MB 542kB/s eta 0:00:01
  Using cached decorator-4.1.2-py2.py3-none-any.whl
  Running setup.py bdist_wheel for hyperopt ... done
  Stored in directory: /home/manuel/.cache/pip/wheels/4b/0f/9d/1166e48523d3bf7478800f250b0fceae31ac6a08b8a7cca820
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /home/manuel/.cache/pip/wheels/27/82/23/785d5d01de2271edf929ac2761cb5dafc0dfa76a0861bfd128
Successfully built hyperopt networkx
  Found existing installation: decorator 4.0.11
    Uninstalling decorator-4.0.11:
      Successfully uninstalled decorator-4.0.11
  Running setup.py install for hpsklearn ... done


In [70]:
from sklearn.metrics import roc_auc_score
from hpsklearn import HyperoptEstimator, random_forest

optimizador_hpsklearn = HyperoptEstimator( classifier=random_forest('estimador_rf'),
                                       seed=42, loss_fn=roc_auc_score, max_evals=100, trial_timeout=30)

In [71]:
%%timeit -n 1 -r 1
optimizador_hpsklearn.fit(censo_X_procesado, censo_y)

25min 59s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


`HyperoptEstimator` nos devuelve un estimador que podemos usar como cualquiera de los de sklearn

In [72]:
optimizador_hpsklearn.predict(censo_X_procesado)

array([0, 0, 0, ..., 0, 0, 0])

con `best_model()` podemos ver el mejor modelo producido

In [74]:
estimador_hpsklearn.best_model()

{'ex_preprocs': (),
 'learner': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=4, max_features=0.011994348367892704,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=3,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1841, n_jobs=1, oob_score=False, random_state=1,
             verbose=False, warm_start=False),
 'preprocs': ()}

Vemos que el estimador de hyperopt-sklearn tambien puede añadir pasos de preprocesamiento automáticos

In [75]:
modelo_hpsklearn = estimador_hpsklearn.best_model()["learner"]

In [76]:
resultados["rf_hpsklearn_10"] = evaluar_modelo(modelo_hpsklearn, censo_X_procesado, censo_y)

In [78]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
naive_bayes,0.066678,0.014363,0.781202,0.783341,0.001043,0.010738,0.849714,0.786865
reg_logistica,0.460645,0.018298,0.906715,0.908326,0.007205,0.013680,0.986234,0.912413
rf,0.395003,0.017742,0.874312,0.995521,0.006179,0.013264,0.950989,1.000000
rf_bayesiano_skopt_100,7.136472,0.193855,0.910383,0.914396,0.111627,0.144932,0.990223,0.918510
rf_gridsearch,3.986320,0.236890,0.848783,0.849026,0.062353,0.177107,0.923221,0.852846
rf_hpsklearn_10,11.909622,0.755976,0.889492,0.890850,0.186288,0.565193,0.967500,0.894859
rf_randomizedsearch,4.077757,0.253586,0.854896,0.855122,0.063783,0.189589,0.929870,0.858970
rf_randomizedsearch_100,63.931399,1.337555,0.919371,0.958161,1.000000,1.000000,1.000000,0.962472
rf_skopt_gp_100,1.329353,0.042163,0.905333,0.909874,0.020793,0.031522,0.984731,0.913968


A modo de prueba, he dejado el Optimizador de hyperopt-sklearn toda la noche corriendo. Lo bueno de este estimador es que en cualquier momento podemos parar el proceso (dandole en el notebook a `kernel->interrup` y el optimizador seguira conservando el modelo mejor.

In [87]:
from hpsklearn import any_classifier, any_preprocessing

estimador_final = HyperoptEstimator( classifier=any_classifier("clf"), 
                                    preprocessing=any_preprocessing("preproc"),
                                    seed=42, loss_fn=roc_auc_score, max_evals=2000)

In [ ]:
estimador_final.fit(censo_X_procesado, censo_y)

In [90]:
estimador_final.best_model()

{'ex_preprocs': (),
 'learner': GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.016925014519856875, loss='deviance',
               max_depth=2, max_features=0.3243309842493083,
               max_leaf_nodes=None, min_impurity_decrease=0.0,
               min_impurity_split=None, min_samples_leaf=1,
               min_samples_split=2, min_weight_fraction_leaf=0.0,
               n_estimators=46, presort='auto', random_state=4,
               subsample=1.0, verbose=0, warm_start=False),
 'preprocs': (PCA(copy=True, iterated_power='auto', n_components=88, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False),)}

In [96]:
modelo_hpsklearn_final = Pipeline([
    ("preprocs", estimador_final.best_model()["preprocs"][0]),
    ("learner", estimador_final.best_model()["learner"])
])

In [97]:
resultados["rf_hpsklearn_final"] = evaluar_modelo(modelo_hpsklearn_final, censo_X_procesado, censo_y)

In [99]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
naive_bayes,0.066678,0.014363,0.781202,0.783341,0.001043,0.010738,0.849714,0.786865
reg_logistica,0.460645,0.018298,0.906715,0.908326,0.007205,0.013680,0.986234,0.912413
rf,0.395003,0.017742,0.874312,0.995521,0.006179,0.013264,0.950989,1.000000
rf_bayesiano_skopt_100,7.136472,0.193855,0.910383,0.914396,0.111627,0.144932,0.990223,0.918510
rf_gridsearch,3.986320,0.236890,0.848783,0.849026,0.062353,0.177107,0.923221,0.852846
rf_hpsklearn_10,11.909622,0.755976,0.889492,0.890850,0.186288,0.565193,0.967500,0.894859
rf_hpsklearn_final,4.160803,0.031582,0.877192,0.880618,0.065082,0.023612,0.954122,0.884580
rf_randomizedsearch,4.077757,0.253586,0.854896,0.855122,0.063783,0.189589,0.929870,0.858970
rf_randomizedsearch_100,63.931399,1.337555,0.919371,0.958161,1.000000,1.000000,1.000000,0.962472
rf_skopt_gp_100,1.329353,0.042163,0.905333,0.909874,0.020793,0.031522,0.984731,0.913968
